In [ ]:
import os
api_key = os.environ.get("OPENAI_API_KEY")

# **Splitting**

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
import json

In [ ]:
# 현재 워킹 디렉토리 설정
current_dir = os.getcwd()

In [18]:
output_dir = os.path.join(current_dir, "output")  # "output" 폴더 경로 설정

pages = []

# output_dir 안의 모든 파일 처리
if os.path.exists(output_dir):  # output 폴더 존재 여부 확인
    for file_name in os.listdir(output_dir):
        if file_name.endswith(".json"):  # JSON 파일
            file_path = os.path.join(output_dir, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                try:
                    data = json.load(f)  # JSON 데이터 로드
                    content = data.get("content", "")  # "content" 키의 값 가져오기
                    if content.strip():
                        # Document 객체로 변환
                        pages.append(Document(page_content=content))
                except json.JSONDecodeError:
                    print(f"파일 {file_name}은 올바른 JSON 형식이 아닙니다.")
else:
    print(f"{output_dir} 디렉토리가 존재하지 않습니다.")

print(f"{len(pages)}개 문서 처리 완료")


109개 문서 처리 완료.


In [4]:
# Text Splitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 문자 수
    chunk_overlap=100  # 청크 간 중복 문자 수
)

In [5]:
docs = text_splitter.split_documents(pages)

# **Embedding**

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [8]:
# OpenAI Embedding 모델 설정
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

/var/folders/pb/tqmxz14n6ps9b507bx6376bh0000gn/T/ipykernel_6510/1742019979.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")


# **Vector Store**

In [9]:
from langchain.vectorstores import Chroma

In [13]:
#Chroma 벡터 스토어 생성 및 저장

db_dir = os.path.join(current_dir, "store")
db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory=db_dir
)

print(f"Chroma 벡터 스토어 저장 완료: {db_dir}")

Chroma 벡터 스토어 저장 완료: /Users/jihoonlim/Desktop/RAG/store


# **Retrieve**

In [14]:
# MMR 기반 검색 설정
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 1,
        "lambda_mult": 0.8
    }
)

In [15]:
# 검색 테스트 및 RAG 시스템에서 사용할 Context 생성
question = "빅뱅이론과 연대측정에 대해 설명해줘."
retrieved_docs = retriever.get_relevant_documents(question)
context = "\n".join([doc.page_content for doc in retrieved_docs])

/var/folders/pb/tqmxz14n6ps9b507bx6376bh0000gn/T/ipykernel_6510/3181881712.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(question)


In [16]:
if retrieved_docs:
    print(f"검색된 문서 개수: {len(retrieved_docs)}")
    print(context)

검색된 문서 개수: 1
생성된 컨텍스트 일부:
사실은 빅뱅 이론에 반대되는 실험 결과나 관측들은 종종 과학적 보고에 제한될 수 있음을 보여주는 것이다.[3] 그러나 이 결과는 전혀 기괴한 것이 아니며, 초기의 비교적 거칠었던 반양성자의 자기모멘트 측정 연구 결과를 다시 한번 확인해주는 것이었다. 빅뱅 이론의 예측과 달리, 이러한 측정은 실험 가능하고, 반복 가능한, 실제 과학이었다. 이 결과는 빅뱅 이론의 신봉자에게만 오직 기이한 것이다.빅뱅 이론으로는 우리의 우주가 존재하지 않아야한다는 결론을 얻은 것은 이번이 처음이 아니다.[4] 빅뱅 과학자들은 그들의 고집스런 주장을 계속하기 위해서는 신뢰할만한 증거들을 제시해야만 한다. 그러나 관측되는 증거들은 빅뱅 이론과 반대된다.빅뱅 과학자들은 빅뱅 이론에 반대되는 그 어떠한 관측 사실도 받아들이지 않는다.빅뱅이 사실이라면 우주가 존재해서는 안 된다는 결론조차도 말이다!빅뱅은 언제나 지지되지 않았던 추측이었다.왜 세속주의자들은 모든 것들의 존재를 창조주를 배제하고 설명하기 위해, 그러
